<a href="https://colab.research.google.com/github/wajihh/-datasciencecoursera/blob/master/rag_gpt4o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Required Libraries

In [ ]:
!pip install qdrant-client langchain transformers pypdf python-docx openai python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.

#Load Envoirnment Variables

In [ ]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env")

# Get API key
openai_api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# Print to confirm (only for testing, avoid in production)
print(f"API Key Loaded: {openai_api_key[:5]}...")  # Just print first few chars


API Key Loaded: sk-AP...


# Upload pdf file

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
from PyPDF2 import PdfReader

# Upload the PDF file
uploaded = files.upload()
# Process file
file_name = list(uploaded.keys())[0]
print(f"Uploaded file: {file_name}")

# Extract text from PDF
reader = PdfReader(file_name)
text = ""
for page in reader.pages:
    text += page.extract_text()

print(f"Extracted {len(text)} characters from the document.")


Saving global_aviation_training.pdf to global_aviation_training.pdf
Uploaded file: global_aviation_training.pdf
Extracted 6107 characters from the document.


#Step 3 Split Documents into Chunks

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# Initialize text splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=50
)

# Split the extracted text into chunks
text_chunks = text_splitter.split_text(text)
print(f"Document split into {len(text_chunks)} chunks.")


Document split into 14 chunks.


#Step 4 Step 4: Generate Embeddings Using a Pre-trained Model

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load HuggingFace embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Generate embeddings for each chunk
embeddings = [embedding_model.embed_query(chunk) for chunk in text_chunks]
print(f"Generated embeddings for {len(embeddings)} chunks.")


<ipython-input-9-db66f38d670e>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generated embeddings for 14 chunks.


\

Step 5: Create a Vector Index in Qdrant

In [ ]:
# Load .env file
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env.qdrant")

True

In [ ]:
from qdrant_client import QdrantClient

# Initialize Qdrant client (replace with your Qdrant URL and API key)
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

# Create a collection in Qdrant
collection_name = "pdf_document_collection"
vector_size = len(embeddings[0])  # Size of each vector from embedding

client.recreate_collection(
    collection_name=collection_name,
    vectors_config={"size": vector_size, "distance": "Cosine"}
)

# Upload document chunks and their embeddings to Qdrant
for i, embedding in enumerate(embeddings):
    client.upload_collection(
        collection_name=collection_name,
        vectors=[embedding],
        payload=[{"text": text_chunks[i], "chunk_index": i}]
    )

print(f"Uploaded {len(embeddings)} chunks to Qdrant.")


<ipython-input-11-4d2b16607d74>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Uploaded 14 chunks to Qdrant.


#Step 6: Query and Retrieve Relevant Chunks

In [ ]:
def query_qdrant(query_text):
    # Generate embedding for the query
    query_embedding = embedding_model.embed_query(query_text)

    # Perform similarity search in Qdrant
    results = client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=3  # Retrieve top 3 matching chunks
    )

    # Display results
    for result in results:
        print(f"Chunk {result.payload['chunk_index']}:\n{result.payload['text']}\n")

# Example query input from the user
user_query = input("Enter your query: ")
query_qdrant(user_query)


Enter your query: list countries 
Chunk 6:
SWEDENEntry Point North AB NordicATS Academy 
TURKEYTurkish Aviation Academy 
UNITED KINGDOMInternational Fire Training Centre (IFTC) Serco 
NATS Training Services 
AFRICA (18)
ALGERIAÉtablissement National de la Navigation Aérienne (CQRENA) 
 
ANGOLAEnana-EP/ Centro Aeronáutico de Instrução  
 
BOTSWANAIAS Aviation Academy 
CAMEROONEcole Régionale de SécuritéIncendie (ERSI) 
ETHIOPIAEthiopian Aviation Academy(EAA) 
Ethiopian Civil Aviation Training Centre (ECATC)

Chunk 7:
Ethiopian Civil Aviation Training Centre (ECATC) 
 JORDANQueen Noor Civil AviationTechnical College (QNCATC) 
KUWAITAustralian College of Kuwait (ACK) 
LEBANONMiddle East Airlines Training Center 
QATARQatar Aeronautical College (QAC) 
SAUDI ARABIASaudi Academy of Civil Aviation (SACA) 
UNITED ARAB EMIRATESGAL ANS Training Centre 
Gulf Center for AviationStudies (GCAS) 
ASIA/PACIFIC (25)
AUSTRALIAASSET Aviation International 
BANGLADESHCivil Aviation Training Centre (CATC)


#Step 7: Synthesize the Response with GPT-4-mini

In [ ]:
!pip install openai

In [ ]:
import openai

# Function to synthesize the final response using GPT-3.5-turbo (accessible on free-tier accounts)
def synthesize_response(query, retrieved_chunks):
    # Combine the query with the retrieved text chunks by accessing the payload correctly
    context = "\n\n".join([chunk.payload['text'] for chunk in retrieved_chunks])

    # Call GPT-3.5-turbo API using the new completions format
    response = openai.completions.create(
        model="gpt-3.5-turbo",  # Switch to gpt-3.5-turbo (which is commonly accessible)
        prompt=f"Based on the following context:\n{context}\nAnswer the query: {query}",
        max_tokens=200
    )

    return response['choices'][0]['text'].strip()

# Perform the search in Qdrant
retrieved_chunks = client.search(
    collection_name=collection_name,
    query_vector=embedding_model.embed_query(user_query),
    limit=3  # Retrieve the top 3 matching chunks
)

# Synthesize the final answer using the retrieved chunks
final_answer = synthesize_response(user_query, retrieved_chunks)
print(f"Final synthesized answer:\n{final_answer}")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}